In [5]:
"""
ac自动机：https://zhuanlan.zhihu.com/p/368184958
什么是AC自动机？
AC自动机(Aho-Corasick automaton)算法于1975年产生于贝尔实验室，
是一种用于解决多模式匹配问题的经典算法。常被用来做敏感词检测，后处理的替换模块也是基于此。
值得注意的是，AC自动机应当属于基于前缀搜索的非压缩字典树。
UNIX之中的grep就是用的这玩意实现的。

AC自动机的原理如下：
关键技术是加入了“失配指针”，可以节省匹配次数，不放弃已匹配过的部分，
AC自动机之中加入了fail路径，又叫失配路径(指针)。
失配指针能够在节点无法匹配下个字符的时候，转向其他节点。
使得算法复杂度比字典树(trie)低，而且跟匹配的字符串长度无关。
缺点是内存开辟大，预加载时间长。

"""

'\nac自动机：https://zhuanlan.zhihu.com/p/368184958\n什么是AC自动机？\nAC自动机(Aho-Corasick automaton)算法于1975年产生于贝尔实验室，\n是一种用于解决多模式匹配问题的经典算法。常被用来做敏感词检测，后处理的替换模块也是基于此。\n值得注意的是，AC自动机应当属于基于前缀搜索的非压缩字典树。\nUNIX之中的grep就是用的这玩意实现的。\n\nAC自动机的原理如下：\n关键技术是加入了“失配指针”，可以节省匹配次数，不放弃已匹配过的部分，\nAC自动机之中加入了fail路径，又叫失配路径(指针)。\n失配指针能够在节点无法匹配下个字符的时候，转向其他节点。\n使得算法复杂度比字典树(trie)低，而且跟匹配的字符串长度无关。\n缺点是内存开辟大，预加载时间长。\n\n'

In [49]:
from collections import defaultdict

class Node(object):
    """
    node
    """
    def __init__(self, str='', is_root=False):
        self._next_p = {}
        self.fail = None
        self.is_root = is_root
        self.str = str
        self.parent = None

    def __iter__(self):
        return iter(self._next_p.keys())

    def __getitem__(self, item):
        return self._next_p[item]

    def __setitem__(self, key, value):
        _u = self._next_p.setdefault(key, value)
        _u.parent = self

    def __repr__(self):
        return "<Node object '%s' at %s>" % \
               (self.str, object.__repr__(self)[1:-1].split('at')[-1])

    def __str__(self):
        return self.__repr__()


class AhoCorasick(object):
    """
    Ac object
    """
    def __init__(self, *words):
        self.words_set = set(words)
        self.words = list(self.words_set)
        self.words.sort(key=lambda x: len(x))
        self._root = Node(is_root=True)
        self._node_meta = defaultdict(set)
        self._node_all = [(0, self._root)]
        _a = {}
        for word in self.words:
            for w in word:
                _a.setdefault(w, set())
                _a[w].add(word)

        def node_append(keyword):
            assert len(keyword) > 0
            _ = self._root
            for _i, k in enumerate(keyword):
                node = Node(k)
                if k in _:
                    pass
                else:
                    _[k] = node
                    self._node_all.append((_i+1, _[k]))
                if _i >= 1:
                    for _j in _a[k]:
                        if keyword[:_i+1].endswith(_j):
                            self._node_meta[id(_[k])].add((_j, len(_j)))
                _ = _[k]
            else:
                if _ != self._root:
                    self._node_meta[id(_)].add((keyword, len(keyword)))

        for word in self.words:
            node_append(word)
        self._node_all.sort(key=lambda x: x[0])
        self._make()


    def _make(self):
        """
        build ac tree
        :return:
        """
        for _level, node in self._node_all:
            if node == self._root or _level <= 1:
                node.fail = self._root
            else:
                _node = node.parent.fail
                while True:
                    if node.str in _node:
                        node.fail = _node[node.str]
                        break
                    else:
                        if _node == self._root:
                            node.fail = self._root
                            break
                        else:
                            _node = _node.fail

    def search(self, content, with_index=False):
        result = set()
        node = self._root
        index = 0
        for i in content:
            while 1:
                if i not in node:
                    if node == self._root:
                        break
                    else:
                        node = node.fail
                else:
                    for keyword, keyword_len in self._node_meta.get(id(node[i]), set()):
                        if not with_index:
                            result.add(keyword)
                        else:
                            result.add((keyword, (index - keyword_len + 1, index + 1)))
                    node = node[i]
                    break
            index += 1
        return result


    def replace(self, kama, kunta):
        '''
        Replace
        '''
        replace_pos = sorted(list(set(self.search(kama, True))))
        for pos_index in replace_pos:
            result = kama[0:pos_index[1][0]] + kunta + kama[pos_index[1][1]:]
        return '{} -> {}'.format(kama, result)


In [51]:
if __name__ == '__main__':
    # ac = AhoCorasick("abc", 'abe', 'acdabd', 'bdf', 'df', 'f', 'ac', 'cd', 'cda')
    # print(ac.search('acdabdf', True))
    ac = AhoCorasick('我爱你', '他们', '你和他', '情趣用品')
    # print(ac.search('我爱你们', True))
    print(ac.replace('我爱你们', ''))
    print(ac.replace('我爱他们', ''))
    print(ac.replace('你和他爱我', ''))
    print(ac.replace('呃，他们爱情趣用品', ''))
    



我爱你们 -> 们
我爱他们 -> 我爱
你和他爱我 -> 爱我
呃，他们爱情趣用品 -> 呃，他们爱
